<a href="https://colab.research.google.com/github/IAM-Hiru/intern/blob/main/INTERNSHIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity

# -----------------------------
# Internship Database
# -----------------------------
internships = pd.DataFrame({
    'InternshipID': [101, 102, 103],
    'CourseName': ['Data Analytics', 'Finance Internship', 'Machine Learning'],
    'CompanyName': ['TechCorp', 'FinanceInc', 'MLLabs'],
    'RequiredSkills': ['Python,SQL', 'Java,Finance', 'ML,Python'],
    'Sector': ['IT', 'Finance', 'IT'],
    'Location': ['Delhi', 'Mumbai', 'Delhi'],
    'PriorityCategory': ['SC', 'OBC', 'ST']
})

# FIXED line
internships['SkillsList'] = internships['RequiredSkills'].str.split(',')

# -----------------------------
# Step 1: User Input
# -----------------------------
print("===== Enter Candidate Details =====")

user_skills = input("Enter your skills (comma separated): ")
user_location = input("Enter your location: ")
user_sector = input("Enter your preferred sector (IT/Finance/etc): ")
user_category = input("Enter your social category (SC/OBC/ST/General): ")
user_past = int(input("Past Participation (0/1): "))

user_skill_list = user_skills.split(',')

# -----------------------------
# Step 2: Skill Encoding
# -----------------------------
all_skills = list(set(sum(internships['SkillsList'].tolist(), []) + user_skill_list))

mlb = MultiLabelBinarizer(classes=all_skills)

internship_matrix = mlb.fit_transform(internships['SkillsList'])
candidate_vector = mlb.transform([user_skill_list])[0]

# -----------------------------
# Step 3: Score Calculation
# -----------------------------
results = []

for i, row in internships.iterrows():
    skill_score = cosine_similarity([candidate_vector], [internship_matrix[i]])[0][0]
    loc_match = int(user_location.lower() == row['Location'].lower())
    sec_match = int(user_sector.lower() == row['Sector'].lower())
    pri_match = int(user_category.upper() == row['PriorityCategory'])

    final_score = (0.5 * skill_score) + (0.2 * loc_match) + (0.2 * sec_match) + (0.1 * pri_match)

    results.append({
        'InternshipID': row['InternshipID'],
        'CourseName': row['CourseName'],
        'CompanyName': row['CompanyName'],
        'FinalScore': final_score
    })

df = pd.DataFrame(results)
df = df.sort_values(by='FinalScore', ascending=False)

# -----------------------------
# Step 4: Output Recommendation
# -----------------------------
print("\n===== BEST INTERNSHIP RECOMMENDATIONS =====")
print(df[['InternshipID', 'CourseName', 'CompanyName', 'FinalScore']])



===== Enter Candidate Details =====
Enter your skills (comma separated): python,sql
Enter your location: dhelli
Enter your preferred sector (IT/Finance/etc): IT
Enter your social category (SC/OBC/ST/General): OBC
Past Participation (0/1): 1

===== BEST INTERNSHIP RECOMMENDATIONS =====
   InternshipID          CourseName CompanyName  FinalScore
0           101      Data Analytics    TechCorp         0.2
2           103    Machine Learning      MLLabs         0.2
1           102  Finance Internship  FinanceInc         0.1
